In [1]:
%matplotlib inline

import time
import scipy.io
import _init_paths
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from utils.logger import Logger
from collections import OrderedDict
from utils.utils import process_config, create_dirs, get_args
from data_loader.data_generator_aug_set import IVUSDataGenerator

e:\miniconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# A function for calculating the Jaccard measure
# By Mehdi Faraji
def Jaccard(Pred, GT):
    intersection = np.sum(Pred & GT)
    union = np.sum(Pred | GT)
    Jacc = intersection / union
    return Jacc

def display_label(idx, img_data, mask, pred):
    img = img_data[idx].reshape((IMG_SIZE, IMG_SIZE))
    mask_gt = mask[idx]
    mask_pred = pred[idx]
    gt = np.multiply(mask_gt, img)
    pred = np.multiply(mask_pred, img) 
    fig, ((ax1), (ax2), (ax3), (ax4), (ax5)) = plt.subplots(nrows=1, ncols=5)
    plt.tight_layout()
    ax1.imshow(img, cmap=plt.get_cmap('gray'))
    ax2.imshow(mask_gt, cmap=plt.get_cmap('gray'))
    ax3.imshow(gt, cmap=plt.get_cmap('gray'))
    ax4.imshow(mask_pred, cmap=plt.get_cmap('gray'))
    ax5.imshow(pred, cmap=plt.get_cmap('gray'))

In [3]:
# change configuration here
IMG_SIZE = 256
subset_size = 20 # feedforward batch size for testing
scale = IMG_SIZE / 512

available_models = range(1, 11)

target = 'Lumen'
# target = 'Media'

augs = ['AM', 'AN', 'AS', 'AZ', 'HM', 'HN', 'HS', 'OC', 'OM', 'ON', 'OS', 'ZI', 'ZN']

# change the model you want to use here
from models.ivusnet import Model

for aug in augs:
    stime = time.time()
    print(aug)

    test_data = np.expand_dims(np.load('../processed_augmentations/{}/test_img_{}.npy'.format(aug, IMG_SIZE)), axis=-1)
    test_mask = np.load('../processed_augmentations/{}/test_{}_512.npy'.format(aug, target.lower())).astype(np.uint8)
    test_subsets = [
        test_data[i:i + subset_size]
        for i in range(0, test_data.shape[0], subset_size)
    ]

    # Reset all for this group of models
    all_result = OrderedDict()
    err = []
    sum_test_result = np.zeros(test_mask.shape)

    configs = []
    # for k in ['prelu-avg']:
    #     for i in available_models:
    #         item = '{}-Journal-ImageSize256-Epoch200-Iter18-LR0.0001-BS6-{}-{}-{}.json'.format(model_version, k, target, i)
    #         configs.append(item)
    for k in ['prelu-max']:
        for i in available_models:
            item = '{}-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-{}-{}-{}.json'.format(aug, k, target, i)
            configs.append(item)

    for conf in configs:
        config = process_config('../configs/{}'.format(conf))
        g_1 = tf.Graph()
        pred_train = []
        pred_test = []
        with g_1.as_default():
            with tf.Session() as sess:
                model = Model(config)
                model.load(sess)
#                 st = time.time()
                for item in test_subsets:
                    pred_test.append(model.predict(sess, item))
#                 print(time.time() - st)
                pred_test = np.vstack(pred_test)
                sum_test_result = sum_test_result + pred_test
                all_result[config.exp_name] = pred_test

    best_score = 0
    best_result = None

    for minus in range(0, len(configs)):
        test_result_ = (sum_test_result >= len(configs) - minus).astype(np.uint8)
#         print('All - {}'.format(minus))
        score = Jaccard(test_result_, test_mask)
#         print(score)
        if score > best_score:
            best_score = score
            best_result = test_result_

    new_result = np.zeros(test_mask.shape)
    c = 0
    for k, v in all_result.items():
        if '-max-' in k:
            new_result = new_result + v
            c += 1

    for minus in range(1, c - 1):
        test_result = (new_result >= c - minus).astype(np.uint8)
#         print('Max - {}'.format(minus))
        score = Jaccard(test_result, test_mask)
#         print(score)
        if score > best_score:
            best_score = score
            best_result = test_result

    new_result = np.zeros(test_mask.shape)
    c = 0
    for k, v in all_result.items():
        if '-avg-' in k:
            new_result = new_result + v
            c += 1

    for minus in range(1, c - 1):
        test_result = (new_result >= c - minus).astype(np.uint8)
#         print('Avg - {}'.format(minus))
        score = Jaccard(test_result, test_mask)
#         print(score)
        if score > best_score:
            best_score = score
            best_result = test_result

    print('Overall best JM result -->', best_score)
    scipy.io.savemat('{}_{}_{}.mat'.format(aug, target, str(best_score)[:6]), 
                     {'{}MaskPredictions'.format(target): best_result})
    print('Time ==>', time.time() - stime)

AM
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_1\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Lumen\checkpoint/-290
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_2\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Lumen\checkpoint/-430
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_3\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Lumen\checkpoint/-330
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_4\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Lumen\checkpoint/-310
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_5\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Lumen\checkpoint/-440
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_6\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Lumen\checkpoint/-300
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_7\AM-Ultrasonic

In [4]:
# change configuration here
IMG_SIZE = 256
subset_size = 20 # feedforward batch size for testing
scale = IMG_SIZE / 512

available_models = range(1, 11)

# target = 'Lumen'
target = 'Media'

augs = ['AM', 'AN', 'AS', 'AZ', 'HM', 'HN', 'HS', 'OC', 'OM', 'ON', 'OS', 'ZI', 'ZN']

# change the model you want to use here
from models.ivusnet import Model

for aug in augs:
    stime = time.time()
    print(aug)

    test_data = np.expand_dims(np.load('../processed_augmentations/{}/test_img_{}.npy'.format(aug, IMG_SIZE)), axis=-1)
    test_mask = np.load('../processed_augmentations/{}/test_{}_512.npy'.format(aug, target.lower())).astype(np.uint8)
    test_subsets = [
        test_data[i:i + subset_size]
        for i in range(0, test_data.shape[0], subset_size)
    ]

    # Reset all for this group of models
    all_result = OrderedDict()
    err = []
    sum_test_result = np.zeros(test_mask.shape)

    configs = []
    # for k in ['prelu-avg']:
    #     for i in available_models:
    #         item = '{}-Journal-ImageSize256-Epoch200-Iter18-LR0.0001-BS6-{}-{}-{}.json'.format(model_version, k, target, i)
    #         configs.append(item)
    for k in ['prelu-max']:
        for i in available_models:
            item = '{}-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-{}-{}-{}.json'.format(aug, k, target, i)
            configs.append(item)

    for conf in configs:
        config = process_config('../configs/{}'.format(conf))
        g_1 = tf.Graph()
        pred_train = []
        pred_test = []
        with g_1.as_default():
            with tf.Session() as sess:
                model = Model(config)
                model.load(sess)
#                 st = time.time()
                for item in test_subsets:
                    pred_test.append(model.predict(sess, item))
#                 print(time.time() - st)
                pred_test = np.vstack(pred_test)
                sum_test_result = sum_test_result + pred_test
                all_result[config.exp_name] = pred_test

    best_score = 0
    best_result = None

    for minus in range(0, len(configs)):
        test_result_ = (sum_test_result >= len(configs) - minus).astype(np.uint8)
#         print('All - {}'.format(minus))
        score = Jaccard(test_result_, test_mask)
#         print(score)
        if score > best_score:
            best_score = score
            best_result = test_result_

    new_result = np.zeros(test_mask.shape)
    c = 0
    for k, v in all_result.items():
        if '-max-' in k:
            new_result = new_result + v
            c += 1

    for minus in range(1, c - 1):
        test_result = (new_result >= c - minus).astype(np.uint8)
#         print('Max - {}'.format(minus))
        score = Jaccard(test_result, test_mask)
#         print(score)
        if score > best_score:
            best_score = score
            best_result = test_result

    new_result = np.zeros(test_mask.shape)
    c = 0
    for k, v in all_result.items():
        if '-avg-' in k:
            new_result = new_result + v
            c += 1

    for minus in range(1, c - 1):
        test_result = (new_result >= c - minus).astype(np.uint8)
#         print('Avg - {}'.format(minus))
        score = Jaccard(test_result, test_mask)
#         print(score)
        if score > best_score:
            best_score = score
            best_result = test_result

    print('Overall best JM result -->', best_score)
    scipy.io.savemat('{}_{}_{}.mat'.format(aug, target, str(best_score)[:6]), 
                     {'{}MaskPredictions'.format(target): best_result})
    print('Time ==>', time.time() - stime)

AM
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_1\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Media\checkpoint/-300
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_2\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Media\checkpoint/-490
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_3\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Media\checkpoint/-80
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_4\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Media\checkpoint/-160
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_5\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Media\checkpoint/-100
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_6\AM-Ultrasonic-ImageSize256-Epoch48-Iter10-LR0.0001-BS4-prelu-max-Media\checkpoint/-130
INFO:tensorflow:Restoring parameters from ../experiments_AM_run_7\AM-Ultrasonic-